In [12]:
import ujson as json
import calendar
import datetime, time
import time as t
import scipy
import numpy as np
start=t.time()
filename='Cedar_Valley.json'
with open(filename, 'r') as f:
    points = json.load(f)
print len(points['features'])

177


In [36]:
jacknife=373236113111401











start_date=1985
end_date=2015
interval=5
resolution=.01
time_tolerance=5
min_ratio=1
min_samples=5
iterations=int((end_date-start_date)/interval+1)

from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
pd.options.display.max_rows=1000
pp = PdfPages('multipage.pdf')
firststart=t.time()
interpolation_df=pd.DataFrame()
combined_df = pd.DataFrame()
for well in points['features']:
    if well['properties']['HydroID']==jacknife or well['properties']['HydroID']==str(jacknife):
        print("found him")
        jacklon=well['geometry']['coordinates'][0]
        jacklat=well['geometry']['coordinates'][1]
    if 'TsTime' in well:
        if len(well['TsTime'])>min_samples and (np.array(well['TsTime']).max()-np.array(well['TsTime']).min())>pd.Timedelta(days=365*5).value/1000000000:
            if np.array(well['TsTime']).max()>calendar.timegm(datetime.datetime(end_date, 1, 1).timetuple()) and np.array(well['TsTime']).min()<calendar.timegm(datetime.datetime(start_date, 1, 1).timetuple()):
                name = str(well['properties']['HydroID'])
            else:
                name = str(well['properties']['HydroID']) + 'Short'
            wells_df = pd.DataFrame(index=pd.to_datetime(well['TsTime'], unit='s', origin='unix'), data=well['TsValue'], columns=[name])
            wells_df.index.drop_duplicates(keep="first")
            wells_df = wells_df.resample('3M').mean()

            combined_df = pd.concat([combined_df, wells_df], join="outer", axis=1)
            combined_df.drop_duplicates(inplace=True)
#combined_df is a pandas dataframe containing the ts depth to water table values for each well in the aquifer with the min number of points
# The columns of combined_df are the HydroIDs of the wells. If the well timeseries data spans the period of interpolation, the column title
# is the HydroID. If the timeseries does not span the time period, then the column title is the HydroID + "Short"
# The rows of the database are the timeseries values for 3 month intervals from start_date to end_date
corrs_df = combined_df.interpolate(method='pchip', limit_area='inside', limit=8)
combined_df.interpolate(method='pchip', inplace=True, limit_area='inside')

#This code resamples the data so we are working on March data since that is what Utah records
combined_df=combined_df.resample('Q').nearest()
decemb=combined_df.index.get_loc(str(start_date)+'-12-31 00:00:00',method='nearest') % 4
combined_df=combined_df.iloc[decemb+1::4]

combined_df['linear']=np.linspace(0,1,len(combined_df))
interpolation_df = combined_df.drop(combined_df.filter(regex='Short').columns, axis=1)
corr_df=corrs_df.corr(min_periods=12)#12
corr_df=corr_df-np.identity(len(corr_df))
length=len(corr_df)-1
for row in corr_df.index:
    if 'Short' in str(row):
        corr_df=corr_df.drop(row)
print t.time()-firststart

well_list=[]
for i in range(length):#length
    reflist=np.array(corr_df.nlargest(5,corr_df.columns[i]).index)
    welli = corr_df.columns[i]
    well_list.append(welli)
    if str(jacknife) in str(welli):
        mydf = combined_df[[welli]].copy()
        print "Well ",welli
        corr_values=corr_df.nlargest(5, corr_df.columns[i]).values[:,i]
        reflist=np.append(reflist,'linear')
        delete_list=[]
        for j in range(len(corr_values)):
            if corr_values[j]<=0.5:
                delete_list.append(j)
        if len(delete_list)>0:
            reflist=np.delete(reflist,delete_list)
        if len(reflist)>=1:
            for ref in reflist:
                mydf=pd.concat([mydf,combined_df[ref]], axis=1)
            norm_exdf=mydf.dropna(how="any", subset=reflist)
            normz_exdf=norm_exdf.copy()
            smaller_exdf=normz_exdf.copy()
            smaller_exdf=smaller_exdf.dropna(how="any")
            for column in norm_exdf.columns[0:]:
                normz_exdf[column]=(norm_exdf[column]-smaller_exdf[column].min())/(smaller_exdf[column].max()-smaller_exdf[column].min())
            exdf = normz_exdf.copy()
            mymin=max(exdf[welli].first_valid_index(),exdf[reflist[0]].first_valid_index())
            mymax="1995-01-01 00:00:00"
            exdf=exdf[mymin:mymax]
            y=exdf[welli]
            x=exdf.as_matrix(columns=reflist)
            n,m=x.shape
            x0=np.ones((n,1))
            x=np.hstack((x,x0))
            model=sm.OLS(y,x)
            model_fit = model.fit_regularized(alpha=0.005)
            b=np.array(model_fit.params)

            A=normz_exdf.as_matrix(columns=reflist)
            n,m=A.shape
            A0=np.ones((n,1))
            A=np.hstack((A,A0))
            norm_exdf['prediction']=np.dot(A,b)
            norm_exdf['prediction']=norm_exdf['prediction']*(norm_exdf[welli].max()-norm_exdf[welli].min())+norm_exdf[welli].min()
            #toggle this next line of code
#             norm_exdf.loc[corrs_df[welli].notnull(),'prediction']=corrs_df[welli]
            newname=str(welli).replace('Short','')
            interpolation_df = pd.concat([interpolation_df, norm_exdf['prediction']], join="outer", axis=1)
            interpolation_df=interpolation_df.rename(columns={"prediction":newname})
            plt.plot(combined_df[welli], 'r', label='Original Data')
            plt.plot(norm_exdf['prediction'], 'r--', label='MLR Prediction')
            plt.plot(combined_df[welli][mymin:mymax], 'g', label='Training Data')
            combined_df[welli+'naive']=combined_df[welli]
            combined_df[welli+'naive'][mymax:]=combined_df[welli+'naive'][combined_df.index.get_loc(mymax,method='nearest')]
            combined_df[welli+'prediction']=norm_exdf['prediction']
            if "Short" in welli:
                well_title="Well "+welli[:-5]
            else:
                well_title="Well "+welli
            plt.title(well_title)
            plt.legend()
            plt.xlabel('Date')
            plt.ylabel('Depth to GW (ft)')
            pp.savefig()
            plt.close()
            test_index=combined_df.index.get_loc('2015-01-01 00:00:00',method='nearest')
            print("MLR")
            print(combined_df[welli+'prediction'][test_index])
            print("Actual")
            print(combined_df[welli][test_index])
pp.close()
plt.show()
print "finished"
print t.time()-firststart

found him
8.25202393532
Well  373236113111401


/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:100: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:108: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

MLR
-49.943681272142946
Actual
-47.56854331861177
finished
9.19168496132


In [ ]:
newinterpolation_df=interpolation_df[str(start_date):str(end_date)].resample('5Y',closed='left').nearest()
corrs_df=corrs_df[str(start_date):str(end_date)].resample('5Y',closed='left').nearest()
corrs_df.rename(columns=lambda x: x.replace('Short',''),inplace=True)
lons=[]
lats=[]
values=[]
elevations=[]
ids=[]
mylon=[]
mylat=[]
myelevs=[]
myids=[]
for wellid in newinterpolation_df.columns:
    for well in points['features']:
        if wellid==str(well['properties']['HydroID']):
            mylon.append(well['geometry']['coordinates'][0])
            mylat.append(well['geometry']['coordinates'][1])
            myelevs.append(well['properties']['LandElev'])
            myids.append(wellid)
            newinterpolation_df.loc[corrs_df[wellid].notnull(),wellid]=corrs_df[wellid]
            break
for i in range(iterations):
    myvalue=np.array(newinterpolation_df.iloc[i].tolist())
    myelev=np.add(np.array(myelevs),myvalue)
    lons.append(mylon)
    lats.append(mylat)
    values.append(myvalue)
    elevations.append(myelev)  
    ids.append(myids)
    print len(mylon)
lons = np.array(lons)
lats = np.array(lats)
ids=np.array(ids)
values = np.array(values)
elevations = np.array(elevations)
print "done"

In [ ]:
import matplotlib.pyplot as plt

nlags=6
#Variogram
def distance(lon1,lat1,lon2,lat2):
    d2=(lon2-lon1)**2+(lat2-lat1)**2
    d=d2**(.5)
    return d
def deviation(value1,value2):
    diff=(value1-value2)**2
    return diff

all_distances=[]
all_deviations=[]
for step in range(0,len(lons)):
    distances=[]
    deviations=[]
    lon=lons[step]
    lat=lats[step]
    value=values[step]
    elevation=elevations[step]
    for x in range(0,len(lon)):
        for y in range(0,len(lon)):
            if x==y:
                continue
            myx=distance(lon[x],lat[x],lon[y],lat[y])
            myy=deviation(value[x],value[y])
            distances.append(myx)
            deviations.append(myy)
    distances=np.array(distances)
    deviations=np.array(deviations)
    max=np.max(distances)
    min=np.min(distances)
    size=((max-min)/4)/nlags

    xbins=[]
    ybins=[]
    meanx=[]
    meany=[]
    
    for c in range(0,(nlags*4)):
        xbin1=[]
        ybin1=[]
        for d in range(0,len(distances)):
            if distances[d]>=(min+size*c) and distances[d]<(min+size*(c+1)):
                xbin1.append(distances[d])
                ybin1.append(deviations[d])
        xbin1=np.array(xbin1)
        ybin1=np.array(ybin1)
        meanx1=np.average(xbin1)
        meany1=np.average(ybin1)
        xbins.append(xbin1)
        ybins.append(ybin1)
        meanx.append(meanx1)
        meany.append(meany1/2)
    
    plt.figure(figsize=(12, 6))
#     plt.scatter(distances, deviations, label='Data')
    plt.scatter(meanx,meany,label='Average')
    plt.show() 
    all_distances.append(distances)
    all_deviations.append(deviations)
all_distances=np.array(all_distances)
all_deviations=np.array(all_distances)
            




In [ ]:
lonmin=360.0
latmin=90.0
lonmax=-360.0
latmax=-90.0
for i in points['features']:
    if i['geometry']['coordinates'][0]<lonmin:
        lonmin=i['geometry']['coordinates'][0]
    if i['geometry']['coordinates'][0]>lonmax:
        lonmax=i['geometry']['coordinates'][0]
    if i['geometry']['coordinates'][1]<latmin:
        latmin=i['geometry']['coordinates'][1]
    if i['geometry']['coordinates'][1]>latmax:
        latmax=i['geometry']['coordinates'][1]
lonmin=round(lonmin-.05,1)
latmin=round(latmin-.05,1)
lonmax=round(lonmax+.05,1)
latmax=round(latmax+.05,1)
print lonmin,lonmax,latmin,latmax
print len(points['features'])

In [ ]:
import numpy as np
latgrid = np.mgrid[latmin:latmax:resolution]
longrid = np.mgrid[lonmin:lonmax:resolution]
latrange=len(latgrid)
lonrange=len(longrid)
print latrange, lonrange


In [ ]:
import pygslib
outx=np.repeat(longrid,latrange)
outy=np.tile(latgrid,lonrange)
grids=[]
for i in range(0,iterations):
    params={
        'x':lons[i],
        'y':lats[i],
        'vr':values[i],
        'nx':lonrange,
        'ny':latrange,
        'nz':1,
        'xmn':lonmin,
        'ymn':latmin,
        'zmn':0,
        'xsiz':resolution,
        'ysiz':resolution,
        'zsiz':1,
        'nxdis':1,
        'nydis':1,
        'nzdis':1,
        'outx':outx,
        'outy':outy,
        'radius':2,
        'radius1':2,
        'radius2':2,
        'ndmax':15,
        'ndmin':5,
        'noct':0,
        'ktype':1,
        'idbg':0,
        'c0':0,
        'it':1,
        'cc':500,
        'aa':.2,
        'aa1':.2,
        'aa2':.2
    }
    estimate= pygslib.gslib.kt3d(params)
    print i
    array=estimate[0]['outidpower']#outest
    array2=estimate[0]['outidpower']
    grid=np.reshape(array,(lonrange,latrange))
    grid2=np.reshape(array2,(lonrange,latrange))
    x=np.isnan(grid)
    grid[x]=grid2[x]
    grids.append(grid)
grids=np.array(grids)

In [ ]:
with open('UtahMajorAquifers.json','r') as f:
    allwells = ''
    wells = f.readlines()
    for i in range(0, len(wells)):
        allwells += wells[i]
utah=json.loads(allwells)


AquiferShape= {
    'type':'FeatureCollection',
    'features':[]
}
print utah['features'][0].viewkeys()
for i in utah['features']:
    if i['properties']['Full_Name']=="Cedar Valley":#Full_Name #AQ_NAME TRINITY
        AquiferShape['features'].append(i)

with open('shapefile.json','w') as outfile:
    json.dump(AquiferShape, outfile)
    

In [ ]:
from __future__ import division
import netCDF4
from netCDF4 import Dataset
import numpy as np
import os
import time as t
from scipy import interpolate
from sklearn.utils import resample
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import numpy as np
import matplotlib.pyplot as plt
import cython
hydros= np.array(ids[0])
volunit="Acre-ft"

interpolation_type="Kriging"
# f=idw(spots,values)
# print f([-96.8,32.7],power=1.0)
# f([-96.0,30.7],power=2,n_neighbors=len(values))

latlen=len(latgrid)
lonlen=len(longrid)
filename='testinterpolate2.nc'
h=netCDF4.Dataset(filename,'w',format="NETCDF4")
h.interpolation="Kriging Multi Linear"
h.start_date=start_date
h.end_date=end_date
h.interval=interval
h.resolution=resolution
h.min_samples=min_samples
h.min_ratio=min_ratio
h.time_tolerance=time_tolerance
h.default=0
h.units='English'


time = h.createDimension("time", 0)
lat = h.createDimension("lat", latlen)
lon = h.createDimension("lon", lonlen)
hydroid=h.createDimension("hydroid", len(ids[0]))
latitude=h.createVariable("lat",np.float64,("lat"))
longitude=h.createVariable("lon",np.float64,("lon"))
time=h.createVariable("time",np.float64,("time"), fill_value="NaN")
hydroids=h.createVariable("hydroid",str,("hydroid"), fill_value="NaN")
hydroids.axis="H"

tsvalue=h.createVariable("tsvalue",np.float64,('time','hydroid'),fill_value=-9999)
tsvalue.units="ft"
tsvalue.coordinates="time hydroid"

depth=h.createVariable("depth",np.float64,('time','lon','lat'),fill_value=-999)
depth.long_name="Depth to Groundwater"
depth.units="ft"
depth.grid_mapping="WGS84"
depth.cell_measures="area: area"
depth.coordinates="time lat lon"

elevation=h.createVariable("elevation",np.float64,('time','lon','lat'),fill_value=-999)
elevation.long_name="Elevation of Groundwater"
elevation.units="ft"
elevation.grid_mapping="WGS84"
elevation.cell_measures="area: area"
elevation.coordinates="time lat lon"

drawdown=h.createVariable("drawdown",np.float64,('time','lon','lat'),fill_value=-999)
drawdown.long_name="Well Drawdown"
drawdown.units="ft"
drawdown.grid_mapping="WGS84"
drawdown.cell_measures="area: area"
drawdown.coordinates="time lat lon"

volume = h.createVariable("volume", np.float64, ('time', 'lon', 'lat'), fill_value=-9999)
volume.long_name = "Change in aquifer storage volume since " + str(start_date)
volume.units = volunit
volume.grid_mapping = "WGS84"
volume.cell_measures = "area: area"
volume.coordinates = "time lat lon"

latitude.long_name="Latitude"
latitude.units="degrees_north"
latitude.axis="Y"
longitude.long_name="Longitude"
longitude.units="degrees_east"
longitude.axis="X"
time.axis="T"
time.units='days since 0001-01-01 00:00:00 UTC'
latitude[:]=latgrid[:]
longitude[:]=longrid[:]
hydroids[:]=ids[0,:]
year=start_date
timearray=[]#[datetime.datetime(2000,1,1).toordinal()-1,datetime.datetime(2002,1,1).toordinal()-1]
            
for i in range(0,iterations):
    print i
    timearray.append(datetime.datetime(year, 1, 1).toordinal() - 1)
    year += interval
    time[i] = timearray[i]
    tsvalue[i,:]=values[i,:]
    for x in range(0,lonrange):
            depth[i,x,:]=grids[i,x,:]
            elevation[i,x,:]=grids[i,x,:]
            if i==0:
                drawdown[i,x,:]=0
            else:
                drawdown[i,x,:]=depth[i,x,:]-depth[0,x,:]

h.close()
os.system("./aquifersubset.sh %s" % (filename))
# serverpath="/home/student/tds/apache-tomcat-8.5.30/content/thredds/public/testdata/groundwater"
# destination= os.path.join(serverpath, filename)
# os.rename(filename, destination)
end=t.time()
print(end-start)



In [ ]:
# os.system("./aquifersubset.sh %s" % (filename))

In [ ]:
import urllib
import copy
from xml.etree import cElementTree as ET
# This function returns the first and last available time from a url of a getcapabilities page located on a Thredds Server
def getTimeBounds(url):
    f=urllib.urlopen(url)
    tree=ET.parse(f)
    root=tree.getroot()
    #These lines of code find the time dimension information for the netcdf on the Thredds server
    dim=root.findall('.//{http://www.opengis.net/wms}Dimension')
    dim=dim[0].text
    times=dim.split(',')
    times.pop(0)
    timemin=times[0]
    timemax=times[-1]
    #timemin and timemax are the first and last available times on the specified url
    return timemin,timemax
#This function returns a pandas dataframe of the timeseries values of a specific layer 
#at a specific latitude and longitude from a file on a Thredds server
#server: the url of the netcdf desired netcdf file on the Thredds server to read
#layer: the name of the layer to extract timeseries information from for the netcdf file
#lat: the latitude of the point at which to extract the timeseries
#lon: the longitude of the point at which to extract the timeseries
#returns df: a pandas dataframe of the timeseries at lat and lon for the layer in the server netcdf file
def getThreddsValue(server,layer,lat,lon):
    #calls the getTimeBounds function to get the first and last available times for the netcdf file on the server
    timemin,timemax=getTimeBounds(server+"?service=WMS&version=1.3.0&request=GetCapabilities")
    #These lines properly format a url request for the timeseries of a speific layer from a netcdf on 
    #a Thredds server
    server=server+"?service=WMS&version=1.3.0&request=GetFeatureInfo&CRS=CRS:84&QUERY_LAYERS="+layer
    server=server+"&X=0&Y=0&I=0&J=0&BBOX="+str(lon)+','+str(lat)+','+str(lon+.001)+','+str(lat+.001)
    server=server+"&WIDTH=1&Height=1&INFO_FORMAT=text/xml"
    server=server+'&TIME='+timemin+'/'+timemax
    f = urllib.urlopen(server)
    tree=ET.parse(f)
    root=tree.getroot()
    features=root.findall('FeatureInfo')
    times=[]
    values=[]
    for child in features:
        time=datetime.datetime.strptime(child[0].text,"%Y-%m-%dT%H:%M:%S.%fZ")
        times.append(time)
        values.append(child[1].text)
    
    df=pd.DataFrame(index=times, columns=[layer],data=values)
    df[layer]=df[layer].replace('none',np.nan).astype(float)
    return df

#function to convert a datetime object to milliseconds since epoch
def datetime_to_float(d):
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    # total_seconds will be in decimals (millisecond precision)
    return total_seconds


server="http://localhost:8080/thredds/wms/testAll/groundwater/testinterpolate2.nc"
layer="depth"
#calls a function to return a df of the timeseries of the pdsi at 37.7,-113
df1=getThreddsValue(server,layer,jacklat,jacklon)
print(df1)
print(jacklat,jacklon)